In [146]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
import cv2
from os import listdir
from os.path import isfile, join
import shutil


In [147]:
tr_d = 'dogs-vs-cats/train/'   #minitrain of 20 images
te_d = 'dogs-vs-cats/randtest/'

onlyfiles = [f for f in listdir('dogs-vs-cats/train/') if isfile(join('dogs-vs-cats/train/', f))]
onlytestfiles = [f for f in listdir('dogs-vs-cats/randtest/') if isfile(join('dogs-vs-cats/randtest/', f))]

trainset = []
for i in range(len(onlyfiles)):
    if i%4 == 0: 
        im = cv2.imread(tr_d + onlyfiles[i], cv2.IMREAD_COLOR)
        im = cv2.resize(im, (80, 80))
        #image = cv2.imread("lenacolor512.tiff", cv2.IMREAD_COLOR)  # uint8 image
        im = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        #print(tr_d + 'cat.' + str(i) + '.jpg')
        #print(im)
        flag = 0
        if onlyfiles[i][:3] == 'dog':
            flag = 1
            #print(i)
        trainset.append([im, flag])
# for i in range(20):
#     im = cv2.imread(tr_d + 'dog.' + str(i) + '.jpg', cv2.IMREAD_COLOR)
#     im = cv2.resize(im, (50, 50))
#     im = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
#    
#     trainset.append([im, 1])
#print(trainset[0][0])
trainset = np.array(trainset)
#print(trainset[0][0])
np.random.shuffle(trainset)
print(trainset.shape)

images = []
labels = []
for i in range(trainset.shape[0]):
    images.append(trainset[i][0])
    labels.append(trainset[i][1])
#print(labels)
images = np.array(images)
labels = np.array(labels)
print(images.shape)
print(labels)
#im = cv2.imread("dogs-vs-cats/minitrain/cat.0.jpg")

#ADD TEST SET

(5750, 2)
(5750, 80, 80, 3)
[1 1 0 ... 1 0 0]


In [148]:
batch_size = 100
b_im = []
b_val = []
for i in range(images.shape[0]//batch_size):
    b_im.append(images[i*batch_size:(i+1)*batch_size])
    b_val.append(labels[i*batch_size:(i+1)*batch_size])

b_im = np.array(b_im)
b_val = np.array(b_val)

print(b_im.shape)

(57, 100, 80, 80, 3)


In [149]:
input_size = 80*80*3
hidden_sizes = [1024, 512]
output_size = 2

model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.LogSoftmax(dim=1))
print(model)

criterion = nn.NLLLoss()

Sequential(
  (0): Linear(in_features=19200, out_features=1024, bias=True)
  (1): ReLU()
  (2): Linear(in_features=1024, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=2, bias=True)
  (5): LogSoftmax()
)


In [150]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) #was 0.004, 0.9
time0 = time()
epochs = 25
for e in range(epochs):
    running_loss = 0
    for images, labels in zip(b_im, b_val):
        images = torch.FloatTensor(images)
        images = images.view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        
        labels = torch.LongTensor(labels)
        loss = criterion(output, labels)
        
        #This is where the model learns by backpropagating
        loss.backward()
        
        #And optimizes its weights here
        optimizer.step()
        
        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss/trainset.shape[0]))
print("\nTraining Time (in minutes) =",(time()-time0)/60)

Epoch 0 - Training loss: 0.006774944129197494
Epoch 1 - Training loss: 0.006539215989734816
Epoch 2 - Training loss: 0.006398838385291722
Epoch 3 - Training loss: 0.006287819561750992
Epoch 4 - Training loss: 0.006197369326715884
Epoch 5 - Training loss: 0.006138625580331554
Epoch 6 - Training loss: 0.006091786550438923
Epoch 7 - Training loss: 0.006027442133944967
Epoch 8 - Training loss: 0.006065825514171434
Epoch 9 - Training loss: 0.0060285511016845705
Epoch 10 - Training loss: 0.0058999953269958496
Epoch 11 - Training loss: 0.005964245376379593
Epoch 12 - Training loss: 0.005886223875957986
Epoch 13 - Training loss: 0.005813477635383606
Epoch 14 - Training loss: 0.005981547842855039
Epoch 15 - Training loss: 0.005877230042996614
Epoch 16 - Training loss: 0.005938295986341394
Epoch 17 - Training loss: 0.005738435791886371
Epoch 18 - Training loss: 0.005904546758402949
Epoch 19 - Training loss: 0.005689750764680946
Epoch 20 - Training loss: 0.005507812474084937
Epoch 21 - Training l

In [151]:
testset = []
for i in range(len(onlytestfiles)):
    im = cv2.imread(te_d + onlytestfiles[i], cv2.IMREAD_COLOR)
    im = cv2.resize(im, (80, 80))
    #image = cv2.imread("lenacolor512.tiff", cv2.IMREAD_COLOR)  # uint8 image
    im = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
    #print(tr_d + 'cat.' + str(i) + '.jpg')
    #print(im)
    flag = 0
    if onlytestfiles[i][:3] == 'dog':
        flag = 1
    testset.append([im, flag])
# for i in range(20):
#     im = cv2.imread(tr_d + 'dog.' + str(i) + '.jpg', cv2.IMREAD_COLOR)
#     im = cv2.resize(im, (50, 50))
#     im = cv2.normalize(im, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
#    
#     trainset.append([im, 1])
#print(trainset[0][0])
testset = np.array(testset)
#print(trainset[0][0])
np.random.shuffle(testset)
print(testset.shape)

images = []
labels = []
for i in range(testset.shape[0]):
    images.append(testset[i][0])
    labels.append(testset[i][1])
#print(labels)
images = np.array(images)
labels = np.array(labels)
print(images.shape)

(2000, 2)
(2000, 80, 80, 3)


In [152]:
t_b_im = []
t_b_val = []
for i in range(images.shape[0]//batch_size):
    t_b_im.append(images[i*batch_size:(i+1)*batch_size])
    t_b_val.append(labels[i*batch_size:(i+1)*batch_size])

t_b_im = np.array(t_b_im)
t_b_val = np.array(t_b_val)

print(t_b_im.shape)

(20, 100, 80, 80, 3)


In [153]:
correct_count, all_count = 0, 0
for images, labels in zip(t_b_im, t_b_val):
        images = torch.FloatTensor(images)
        #print(t_b_im.shape)
        images = images.view(images.shape[0], -1)
        #print(images.shape)
        # Training pass
        with torch.no_grad():
            logps = model(images)
        #print(logps.shape, logps[0])
        ps = torch.exp(logps)
        #print(ps.shape)
        for i in range(batch_size):
            prob_ans = ps[i]
            prob_ans = list(prob_ans)
            pred_label = prob_ans.index(max(prob_ans))
            #pred_label = ps.index(max(ps))
            true_label = labels[i]
            if(true_label == pred_label):
              correct_count += 1
            all_count += 1

In [154]:
print(all_count)
print(correct_count/all_count)

2000
0.613
